In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from xfem import *
from xfem.lsetcurv import *

SetHeapSize(100000000)

# -------------------------------- PARAMETERS ---------------------------------
# Mesh size
maxh = 0.125
# Finite element order (k of P^k/P^{k-1} Taylor-Hood elements)
order = 3

# Problem parameters
mu = [1, 10]

R = 2.0 / 3.0

aneg = 1.0 / mu[0]
apos = 1.0 / mu[1] + (1.0 / mu[0] - 1.0 / mu[1]) * exp(x * x + y * y - R * R)

# Discretisation parameters

# Stabilization parameters for ghost-penalty
gamma_stab_v = 0.05  # if set to zero: no GP-stabilization for velocity
gamma_stab_p = 0.05
# Stabilization parameter for Nitsche
lambda_nitsche = 0.5 * (mu[0] + mu[1]) * 20 * order * order

# ----------------------------------- MAIN ------------------------------------
# Generate the background mesh
square = SplineGeometry()
square.AddRectangle((-1, -1), (1, 1), bcs=[1, 2, 3, 4])
mesh = Mesh(square.GenerateMesh(maxh=maxh, quad_dominated=False))

d = mesh.dim


# Construct the exact level set function, radius and radius squared:
rsqr = x**2 + y**2
r = sqrt(rsqr)

levelset = r - R
Draw(mesh)

importing ngsxfem-2.1.2405


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [2]:
# Construct the exact solution:
gammaf = 0.5  # surface tension = pressure jump
vel_exact = [a * CoefficientFunction((-y, x)) * exp(-1 * rsqr)
             for a in [aneg, apos]]
pre_exact = [x**3, x**3 - gammaf]


# Some helper expressions to compute the r.h.s. of the Stokes system for
# a given solution:
def Coef_Grad(v):
    func = tuple([v[i].Diff(w) for w in [x, y] for i in [0, 1]])
    return CoefficientFunction(func, dims=(d, d)).trans


def Coef_Eps(v):
    return 0.5 * (Coef_Grad(v) + Coef_Grad(v).trans)


def Coef_Div(v):
    func = tuple([sum([v[i, j].Diff(w) for j, w in [(0, x), (1, y)]])
                  for i in range(d)])
    return CoefficientFunction(func)


# Manufactured right-hand side:
coef_g = [Coef_Div(-2 * mu[i] * Coef_Eps(vel_exact[i])
                   + pre_exact[i] * Id(d)) for i in [0, 1]]

